# Name entities recognition on each intent of the dataset

In [1]:
dyn.load(paste0(system2('/usr/libexec/java_home', stdout = TRUE), '/jre/lib/server/libjvm.dylib'))
library(NLP)
library(openNLP)
library(RWeka)
library(rJava)
library(magrittr)

Warning message:
“package ‘RWeka’ was built under R version 3.4.3”Warning message:
“package ‘rJava’ was built under R version 3.4.2”

In [2]:
#install.packages("openNLPmodels.en",
#                 repos = "http://datacube.wu.ac.at/",
#                 type = "source")

In [3]:
citations <- read.csv(file="opinions_sample_after_scraping.csv", header=TRUE, sep=";")

In [4]:
head(citations)

X,case_cite,parties,docket,court,date,intent,id


In [5]:
citation <- citations[3,]
#citation <- "Carousel is the second musical by the team of Richard Rodgers (music) and Oscar Hammerstein II (book and lyrics). The 1945 work was adapted from Ferenc Molnár's 1909 play Liliom, transplanting its Budapest setting to the Maine coastline. The story revolves around carousel barker Billy Bigelow, whose romance with millworker Julie Jordan comes at the price of both their jobs. He participates in a robbery to provide for Julie and their unborn child; after it goes tragically wrong, he is given a chance to make things right. A secondary plot line deals with millworker Carrie Pipperidge and her romance with ambitious fisherman Enoch Snow. The show includes the well-known songs \"If I Loved You\", \"June Is Bustin\' Out All Over\" and \"You'll Never Walk Alone\". Richard Rodgers later wrote that Carousel was his favorite of all his musicals."
intent <- paste(citation$intent, collapse = "")
#intent <- as.String(citation)
intent

[1] "['Arthur Brown and John H. Mitchell, for plaintiff in error.', 'Asst. Atty. Gen. Maury, for defendant in error.', 'Mr. Justice HARLAN, after stating the facts in the foregoing language, delivered the opinion of the court.', \"The Penal Code of Utah, established by the act of February 18, 1876, provides that 'every murder perpetrated by poison, lying in wait, or any other kind of willful, deliberate, malicious, and premeditated killing; or committed in the perpetration of, or attempt to perpetrate, any arson, rape, burglary, or robbery; or perpetrated from a premeditated design, unlawfully and maliciously to effect the death of any other human being other than him who is killed; or perpetrated by any act greatly dangerous to the lives of others, and evidencing a depraved mind, regardless of human life,—is murder in the first degree; and any other homicide, committed under such circumstances as would have constituted murder at common law, is murder in the second degree.' Comp. Laws Utah 1876, p. 585. The same Code further provides that 'every person guilty of murder in the first degree shall suffer death, or, upon the recommendation of the jury, may be imprisoned at hard labor in the penitentiary for life, at the discretion of the court, and every person guilty of murder in the second degree shall be imprisoned at hard labor and the penitentiary for not less than five nor more than fifteen years.' Comp. Laws Utah 1876, p. 586. It is clear that the authority given in the section last quoted, to substitute imprisonment at hard labor in the penitentiary for life for the penalty of death, when the accused is found guilty of murder in the first degree, depends upon a previous recommendation to that effect by the jury. Without such recommendation, the court, in the absence of sufficient grounds for a new trial, has no alternative but to sentence the accused to suffer death. While in this case the jury were instructed as to what constituted murder in the first and second degrees, they were not informed as to their right, under the statute, to recommend imprisonment for life at hard labor in the penitentiary in place of the punishment of death. If their attention had been called to that statute, it may be that they would have made such a recommendation, and thereby enabled the court to reduce the punishment to imprisonment for life. We are of opinion that the court erred in not directing the attention of the jury to this matter. The statute evidently proceeds upon the ground that there may be cases of murder in the first degree, the punishment for which by imprisonment for life at hard labor will suffice to meet the ends of public justice. Its object could only have been met through a recommendation by the jury that the lesser punishment be inflicted, and it is not to be presumed that they were aware of their right to make such recommendation. The failure of the court to instruct them upon this point prevented it from imposing the punishment of imprisonment for life, even if, in its judgment, the circumstances of the case rendered such a course proper. It was well said in the dissenting opinion of Mr. Justice HENDERSON, in the supreme court of the territory, that by the action of the district court 'the prisoner was deprived of a substantial right. The determination of the question as to whether he should suffer death or imprisonment was one of vital consequence to him. The jury, to whom the statute commits the determination of that question, at least in part, were not informed of their duty and responsibility in the matter, so as to require them to exercise their judgment and discretion in relation to it, and, by the verdict they rendered, the court had none.' These views are in accordance with the fundamental rules obtaining in the trial of criminal cases involving life.\", 'Other questions were discussed at the bar, but as the instructions relating to them are somewhat obscure, and as they may not arise upon another trial in the

In [6]:
word_ann <- Maxent_Word_Token_Annotator()
sent_ann <- Maxent_Sent_Token_Annotator()

In [7]:
intent_annotations <- annotate(intent, list(sent_ann, word_ann))
head(intent_annotations)
intent_doc <- AnnotatedPlainTextDocument(intent, intent_annotations)
sents(intent_doc) %>% head(2)
words(intent_doc) %>% head(10)

 id type     start end features
  1 sentence     1  62 constituents=<<integer,14>>
  2 sentence    64  69 constituents=<<integer,3>>
  3 sentence    71  75 constituents=<<integer,2>>
  4 sentence    77 113 constituents=<<integer,11>>
  5 sentence   115 118 constituents=<<integer,2>>
  6 sentence   120 223 constituents=<<integer,21>>

[[1]]
 [1] "['Arthur"  "Brown"     "and"       "John"      "H."        "Mitchell" 
 [7] ","         "for"       "plaintiff" "in"        "error"     "."        
[13] "'"         ","        

[[2]]
[1] "'"    "Asst" "."

[1] "['Arthur"  "Brown"     "and"       "John"      "H."        "Mitchell" 
 [7] ","         "for"       "plaintiff" "in"

In [10]:
person_ann <- Maxent_Entity_Annotator(kind = "person")
location_ann <- Maxent_Entity_Annotator(kind = "location")
organization_ann <- Maxent_Entity_Annotator(kind = "organization")
date_ann <- Maxent_Entity_Annotator(kind = "date")
#misc_ann <- Maxent_Entity_Annotator(kind = "misc")


In [12]:
intent_annotations <- annotate(intent, list(sent_ann, word_ann, person_ann, location_ann, organization_ann, date_ann))
intent_doc <- AnnotatedPlainTextDocument(intent, intent_annotations)

In [13]:
entities <- function(doc, kind) {
  s <- doc$content
  a <- annotations(doc)[[1]]
  if(hasArg(kind)) {
    k <- sapply(a$features, `[[`, "kind")
    s[a[k == kind]]
  } else {
    s[a[a$type == "entity"]]
  }
}

In [15]:
entities(intent_doc, kind = "person")
entities(intent_doc, kind = "location")
entities(intent_doc, kind = "organization")
entities(intent_doc, kind = "date")

[1] "Brown"            "John H. Mitchell" "Maury"

character(0)

character(0)

[1] "February 18, 1876" "1876"              "1876"